In [3]:
import requests

from tqdm.auto import tqdm
from openai import OpenAI
from elasticsearch import Elasticsearch

### Getting the data

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Alternative answer for Question 01

In [6]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

{'name': 'b400d59e3fa3',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'aoGHGEnYQFOXpxGFr1Jcgw',
 'version': {'number': '8.4.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
  'build_date': '2022-10-04T07:17:24.662462378Z',
  'build_snapshot': False,
  'lucene_version': '9.3.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

Question 02

In [16]:
from elasticsearch import RequestError

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

try:
    es_client.indices.create(index=index_name, body=index_settings)
    for doc in tqdm(documents):
        es_client.index(index=index_name, body=doc)
except RequestError as e:
    if "resource_already_exists_exception" in str(e):
        print(str(e.info['error']['reason']))
    else: 
        raise e

index [course-questions/HX_wpqlWQlWVIBSl4toD3w] already exists


### Question 03

In [23]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [24]:
response

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 865, 'relation': 'eq'},
  'max_score': 84.050095,
  'hits': [{'_index': 'course-questions',
    '_id': 'WFWcS5ABOlwbx5xOjJ-3',
    '_score': 84.050095,
    '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
     'section': '5. Deploying Machine Learning Models',
     'question': 'How do I debug a docker container?',
     'course': 'machine-learning-zoomcamp'}},
   {'_index': 'course-questions',
    '_id': 'ZlWcS5ABOlwbx5xOiZ3N',
    '_score': 75.54128,
    '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you c

### Question 04

In [50]:
query = "How do I execute a command in a running docker container?"
search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    },
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(index=index_name, body=search_query)

In [51]:
response

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 345, 'relation': 'eq'},
  'max_score': 84.050095,
  'hits': [{'_index': 'course-questions',
    '_id': 'WFWcS5ABOlwbx5xOjJ-3',
    '_score': 84.050095,
    '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
     'section': '5. Deploying Machine Learning Models',
     'question': 'How do I debug a docker container?',
     'course': 'machine-learning-zoomcamp'}},
   {'_index': 'course-questions',
    '_id': 'd1WcS5ABOlwbx5xOjJ_h',
    '_score': 51.04628,
    '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp comm

### Question 05

In [52]:
def build_context(search_results):
    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    context = ""
    for hit in search_results['hits']['hits']:
        source_ = hit['_source']
        context += f"{context_template.format(question=source_['question'], text=source_['text'])}\n\n"

    return context

In [53]:
context = build_context(response)
print(context)

Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
    A: You can copy files from your local machine into a Docker container using the docker cp command. He

In [54]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [55]:
prompt = prompt_template.format(question=query, context=context)

In [56]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
    A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory conta

In [57]:
len(prompt)

1476

### Question 06

In [58]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 95.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 790.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 3.8 MB/s eta 0:00:0000:0100:01


In [60]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [62]:
tokens = encoding.encode(prompt)

In [63]:
len(tokens)

326

In [64]:
encoding.decode_single_token_bytes(63842)

b"You're"

### Question 07

In [65]:
client = OpenAI()

response = client.chat.completions.create(
    model='gpt-4o', 
    messages=[{'role': "user", 'content': prompt}]
)

response.choices[0].message.content

'To execute a command in a running Docker container, you need to follow these steps:\n\n1. Identify the container ID by running:\n   ```sh\n   docker ps\n   ```\n\n2. Execute a command inside the specific container using the `docker exec` command. For example, to start a bash session in the container:\n   ```sh\n   docker exec -it <container-id> bash\n   ```\n   Replace `<container-id>` with the actual ID of the running container.\n\n(Marcos MJD)'

### Question 08

In [66]:
output_tokens = encoding.encode(response.choices[0].message.content)

In [67]:
len(output_tokens)

104

In [70]:
cost = ((len(tokens) * 0.005) + (len(output_tokens) * 0.015)) / 1000

In [71]:
cost

0.0031900000000000006